In [1]:
import torch
from itertools import combinations
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from src import regression
from src.data_manager import *

In [2]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [3]:
norm = Normalizer()
X_n = norm.fit(X)

In [4]:
# norm.inverse_transform(X_n).flatten() == X.flatten()

In [6]:
prim = Primitives()
data = prim.transform(X, y)

In [7]:
data = data[:, ~data.isnan().all(dim=0)]
data.shape

torch.Size([20640, 54])

In [ ]:
betas = []
intercepts = []
for i1, i2 in combinations(range(data.shape[1]), 2):
    model = LinearRegression()
    model.fit(X, y)
    betas.append(model.coef_)
    intercepts.append(model.intercept_)
    # X_aug = torch.cat(
    #     [
    #         torch.ones(X.shape[0], 1), 
    #         X
    #     ], 
    #     dim=1
    # )
    # theta, *_ = torch.linalg.lstsq(X_aug, y)
    # betas.append(theta[1:].cpu())
    # intercepts.append(theta[0].cpu())
    # del theta, X, X_aug
    # torch.cuda.empty_cache()
#     # break

In [9]:
model = regression.PairwiseLinearRegression()

In [10]:
model.fit(data, y)

In [11]:
torch.cuda.empty_cache()

In [12]:
preds = model.predict(data)

torch.Size([20640, 1431]) torch.Size([20640, 54])


In [13]:
scores = model.evaluate(data, y, ["mape"])

torch.Size([20640, 1431]) torch.Size([20640, 54])


In [14]:
scores['mape'].min()

tensor(0.3555)